In [1]:
import glob

train_dir = '../input/crop_train'
test_dir = '../input/test'
test_files = sorted(glob.glob(test_dir+'/*'))
train_files = sorted(glob.glob(train_dir+'/*/*'))
from keras.models import load_model
model = load_model('best_xception_manip.h5')

Using TensorFlow backend.


In [2]:
import os
import numpy as np
from PIL import Image
import skimage
import skimage.io
import scipy.misc
import imageio
import warnings; warnings.simplefilter('ignore')
CROP_LEN = 112


def center_crop(im_array):
    center_x = im_array.shape[0] // 2
    center_y = im_array.shape[1] // 2
    half_crop = CROP_LEN // 2
    a = center_x - half_crop
    b = a + CROP_LEN
    c = center_y - half_crop
    d = c + CROP_LEN
    new_im_array = im_array[a:b,c:d,:]
    return new_im_array


def get_tta_img(img_path):
    im_array = np.array(Image.open((img_path)), dtype="uint8")
    img_x = im_array.shape[0]
    img_y = im_array.shape[1]
    im_array = im_array/255.0
    
    img_list = []
    img_list.append(center_crop(im_array))
    img_list.append(im_array[0:CROP_LEN,0:CROP_LEN,:])
    img_list.append(im_array[0:CROP_LEN,img_y-CROP_LEN:img_y,:])
    img_list.append(im_array[img_x-CROP_LEN:img_x,0:CROP_LEN,:])
    img_list.append(im_array[img_x-CROP_LEN:img_x,img_y-CROP_LEN:img_y,:])
    return np.array(img_list)
print('def done')

def done


In [3]:
list_classes = [
 'Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

tmp_cnt = 0
x_data,y_data = [],[]
for img_p in train_files:
    tmp_cnt += 1
    if tmp_cnt % 500 == 0:
        print('processing',tmp_cnt)
    
    tta_imgs = get_tta_img(img_p)
    if 'manip.jpg' in img_p:
        manip_v = 1
    else:
        manip_v = 0
    
    # add x,y
    res = list(model.predict(tta_imgs).flatten()) + [manip_v]
    x_data.append(res)
    tmp_y = img_p.split('/')[-2]
    y_data.append(list_classes.index(tmp_y))

print('train_x done')
print(x_data[0])
print(y_data[0])

processing 500
processing 1000
processing 1500
processing 2000
processing 2500
processing 3000
processing 3500
processing 4000
processing 4500
processing 5000
processing 5500
processing 6000
processing 6500
processing 7000
processing 7500
processing 8000
processing 8500
processing 9000
processing 9500
processing 10000
processing 10500
processing 11000
processing 11500
processing 12000
processing 12500
processing 13000
processing 13500
processing 14000
processing 14500
processing 15000
processing 15500
processing 16000
processing 16500
processing 17000
processing 17500
processing 18000
processing 18500
processing 19000
processing 19500
processing 20000
processing 20500
processing 21000
processing 21500
processing 22000
processing 22500
processing 23000
processing 23500
processing 24000
processing 24500
processing 25000
processing 25500
processing 26000
processing 26500
processing 27000
processing 27500
processing 28000
processing 28500
processing 29000
processing 29500
processing 30000


In [4]:
tmp_cnt = 0
test_x_data = []
for img_p in test_files:
    tmp_cnt += 1
    if tmp_cnt % 500 == 0:
        print('processing',tmp_cnt)
    
    tta_imgs = get_tta_img(img_p)
    if 'manip' in img_p:
        manip_v = 1
    else:
        manip_v = 0
    
    # add x,y
    res = list(model.predict(tta_imgs).flatten()) + [manip_v]
    test_x_data.append(res)

print('test x done')
print(test_x_data[0])



processing 500
processing 1000
processing 1500
processing 2000
processing 2500
test x done
[0.02359141, 0.061537307, 0.0019089385, 0.01738517, 0.016727671, 0.0013747044, 0.33348948, 0.14577962, 0.04280499, 0.35540065, 0.0012152579, 0.33947885, 0.011290116, 0.082711026, 0.01061673, 9.2737217e-05, 0.042696849, 0.15168965, 0.011955949, 0.3482528, 0.00016892055, 0.4249813, 0.0058657425, 0.17064752, 0.015824525, 5.1346091e-05, 0.0046114125, 0.16277969, 0.050405115, 0.16466445, 2.8553446e-05, 0.01642832, 0.00090290239, 0.074985228, 0.010228889, 0.0007561784, 0.18931058, 0.5130145, 0.001205222, 0.19313964, 0.00014413751, 0.057914909, 0.00035841385, 0.20828924, 0.0063903141, 0.00015522163, 0.032842327, 0.29411545, 0.015266438, 0.38452363, 1]


In [5]:
import pickle
with open('../features/x_feat.pkl','wb') as fout:
    pickle.dump([x_data,y_data,test_x_data],fout)
print('feat done')

feat done
